# 和 user 確認問題，持續修改重點直到 user verify

In [ ]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')
print(OpenAI.api_key)

In [ ]:
instructions = [
    "As focus app 'FLORA' intelligent customer service chatbot, my goal is to assist you effectively. ",
    "To ensure accuracy, I'll confirm your input and distill it into key points. Then, I will and must list these points using bullet form and confirm with you if they're correct. ",
    "If you have alternative descriptions for any key point, I'll update the description accordingly. ",
    "We'll repeat this process until you're satisfied. ",
]

instruction = ""
for i in instructions:
    instruction += i

print(instruction)

In [ ]:
assistant = client.beta.assistants.create(
    name = "Intelligent customer service chatbot",
    instructions = instruction,
    model = "gpt-3.5-turbo-1106"
)

In [ ]:
thread = client.beta.threads.create()

# Chat start from here

In [ ]:
user_input = ""

In [ ]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = user_input
)

In [ ]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)

while run.status == "queued" or run.status == "in_progress":
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )
print(run.status)

In [ ]:
messages = client.beta.threads.messages.list(
    thread_id = thread.id,
    order = 'asc'
)

for i in messages.data[-10:]:
    print(i.content[0].text.value)
    print("==========")

# Keep key point

In [ ]:
verified_content = messages.data[-1].content[0].text.value
print(verified_content)

In [ ]:
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "Retain only the key points. Ensure that the retained key points remain unchanged and intact. Delete any content that is not included in the key points."},
        {"role": "user", "content": "Retain only the key points. Ensure that the retained key points remain unchanged and intact. Delete any content that is not included in the key points: \n" + verified_content}
    ]
)
print(completion.choices[0].message.content)

# Delete Assistant

In [ ]:
client.beta.assistants.delete(assistant.id)

In [ ]:
client.beta.assistants.list(
    order="desc",
    limit="20",
)